In [1]:
import math
import numpy as np
import pandas as pd



In [2]:
df_fin = pd.read_csv('./pickle_files/df_final_spark_top5k_log_cv_160421.csv',index_col=0)

In [3]:
df_fin_true = df_fin[df_fin['has_enough_stat']]

In [4]:
df_fin_true.head()


,item_code,avg_conv_rate,last_price,last_date,avg_unique_views,PE,PE_err,has_enough_stat,pc2,item_skey
2,1000021922,0.023490,114.99,2021-02-16,90.113095,-4.0089,0.5605,True,NaN,156995084198890
3,1000022179,0.003121,479.99,2021-02-06,93.258739,-3.5759,1.1543,True,276.870,11954970507506
10,1000145006,0.004873,289.99,2021-02-04,188.190476,-4.6079,1.3188,True,188.819,280637889289282
14,1000005727,0.017079,99.99,2021-02-25,170.099034,-1.5183,0.4891,True,40.729,71488185050378
17,1000008870,0.001093,619.99,2021-02-11,130.857143,-3.5079,1.1074,True,259.209,242371880591990


In [5]:
df_final_spark=df_fin_true.astype({'pc2': 'float64'})

print(df_final_spark.dtypes)

df_final_spark.dropna(inplace=True)
df_final_spark = df_final_spark[df_final_spark['PE']< 1.0]
df_final_spark = df_final_spark[df_final_spark['PE_err']<1.0]

item_code             int64
avg_conv_rate       float64
last_price          float64
last_date            object
avg_unique_views    float64
PE                  float64
PE_err              float64
has_enough_stat        bool
pc2                 float64
item_skey             int64
dtype: object


In [6]:
VAT = 1.19



In [7]:
df_n_views = pd.read_csv('./../raw_data/Excel_files/Top5k_View_PE.csv',index_col = 0)

In [8]:
df_n_views.head()

,View_PE,View_PE_err
1000012530,-1.6265,0.4565
1000021922,-1.0147,0.1548
1000034762,-1.3782,0.3497
1000125896,-1.6598,0.3366
1000005727,-0.9853,0.2722


In [9]:
df_n_views['item_code'] = df_n_views.index

In [10]:
df_n_views.head()

,View_PE,View_PE_err,item_code
1000012530,-1.6265,0.4565,1000012530
1000021922,-1.0147,0.1548,1000021922
1000034762,-1.3782,0.3497,1000034762
1000125896,-1.6598,0.3366,1000125896
1000005727,-0.9853,0.2722,1000005727


In [11]:
df_merged = pd.merge(df_final_spark,df_n_views, on="item_code")

In [12]:
df_merged.head()


,item_code,avg_conv_rate,last_price,last_date,avg_unique_views,PE,PE_err,has_enough_stat,pc2,item_skey,View_PE,View_PE_err
0,1000005727,0.017079,99.99,2021-02-25,170.099034,-1.5183,0.4891,True,40.729,71488185050378,-0.9853,0.2722
1,1000016028,0.016290,139.99,2021-02-25,26.446154,-2.3105,0.5174,True,52.430,48800168481821,-1.6602,0.2891
2,1000175476,0.055748,83.99,2021-02-25,80.722222,-2.0583,0.2891,True,48.799,115260611693891,-0.5003,0.1561
3,1000176734,0.007736,159.99,2021-02-08,100.555556,-1.7633,0.5678,True,54.119,38255991794505,-1.2500,0.3075
4,1000194074,0.024822,77.99,2021-02-25,97.437774,-1.5244,0.4274,True,43.439,242522860992899,-0.9045,0.2371


In [13]:
df_merged['PE'].mean()

-2.4940368715083796

In [14]:
def create_auxiliary_var(dataf=df_merged):
    dataf['Delta_price'] = (dataf['last_price']/VAT - dataf['pc2'])
    dataf['Margin_base'] = dataf['avg_unique_views']*dataf['avg_conv_rate']*(dataf['Delta_price'])

def classify(dataf= df_merged):
    dataf['Group_flag'] = np.where(dataf['Margin_base'] < (dataf['Margin_up'] - 3*dataf['Margin_err_up']), 'up',
                                        np.where(dataf['Margin_base'] < (dataf['Margin_down'] - 3*dataf['Margin_err_down']), 'down','neutral'))

def classify_up(dataf=df_merged):
    dataf['Group_flag'] = np.where(dataf['Margin_base'] < (dataf['Margin_up'] - 3*dataf['Margin_err_up']), 'up','neutral_up')

def classify_down(dataf=df_merged):
    dataf['Group_flag'] = np.where(dataf['Margin_base'] < (dataf['Margin_down'] - 3*dataf['Margin_err_down']), 'down','neutral_down')

In [52]:
def create_Margin( increment = 0.01, dataf = df_merged, flag = 'up'):
    if flag == 'down':
        increment = -increment

    price          = 'price_'+flag
    Delta_price    = 'Delta_price_'+flag
    conv_rate      = 'conv_rate_'+flag
    conv_rate_err  = 'conv_rate_err_'+flag
    n_views        = 'avg_unique_views_' + flag
    n_views_err    = 'avg_unique_views_err_' + flag
    Margin         = 'Margin_'+flag
    Margin_err     = 'Margin_err_'+flag

    #CROSS CHECK THE WHOLE ERROR PROPAGATION!!!!

    dataf[price]              =  dataf['last_price']*math.exp(increment)
    dataf[Delta_price]        =  (dataf[price]/VAT - dataf['pc2'])
    dataf[conv_rate]          =  dataf['avg_conv_rate']*np.exp(dataf['PE']*np.log(dataf[price]/dataf['last_price']))
    dataf[conv_rate_err]      =  np.fabs(dataf[conv_rate])*np.fabs(dataf['PE_err']*np.log(dataf[price]/dataf['last_price']))
    dataf[n_views]            =  dataf['avg_unique_views']*np.exp(dataf['View_PE']*np.log(dataf[price]/dataf['last_price']))
    dataf[n_views_err]        =  np.fabs(dataf[n_views])*np.fabs(dataf['View_PE_err']*np.log(dataf[price]/dataf['last_price']))
    dataf[Margin]             =  dataf[n_views]*dataf[conv_rate]*(dataf[Delta_price])
    #Propagate Margin error
    dataf[Margin_err]         =  np.sqrt(np.power(dataf[conv_rate]*dataf[n_views_err]*(dataf[Delta_price]),2) + np.power(dataf[n_views]*dataf[conv_rate_err]*(dataf[Delta_price]),2))



In [53]:
#print(df_final_spark[['item_code','avg_conv_rate','last_price']].head())
#create_Margin(0.01)

In [54]:
original_df= df_merged
print(len(original_df.index))
tmp_df = pd.DataFrame()

original_df_up   = pd.DataFrame()
original_df_down = pd.DataFrame()

358


In [55]:
list_of_columns_up   = ['item_code','last_date','last_price','price_up','PE','PE_err','View_PE','View_PE_err','Delta_price','pc2','avg_conv_rate','avg_unique_views','avg_unique_views_up','Margin_base','Margin_up','Group_flag']
list_of_columns_down = ['item_code','last_date','last_price','price_down','PE','PE_err','View_PE','View_PE_err','Delta_price','pc2','avg_conv_rate','avg_unique_views','avg_unique_views_down','Margin_base','Margin_down','Group_flag']
create_auxiliary_var(original_df)
for inc in [0.01,0.2,0.3,0.4]:#,0.3,0.4,0.5]: #np.linspace(0.01,0.1,10):
    print('Increment = ' + str(inc))
    if inc == 0.01:
        create_Margin(inc,original_df,'up')
        create_Margin(inc,original_df,'down')
        classify(original_df)
        tmp_df           = tmp_df.append(original_df[original_df['Group_flag'] == 'neutral'])
        original_df_up   = original_df_up.append(original_df[original_df['Group_flag'] == 'up'])
        original_df_up   = original_df_up[list_of_columns_up]
        original_df_down = original_df_down.append(original_df[original_df['Group_flag'] == 'down'])
        original_df_down   = original_df_down[list_of_columns_down]
        #print('tmp\n',tmp_df[list_of_columns].head())
        #print('up\n', original_df_up[list_of_columns].head())
        #print('down\n',original_df_down[list_of_columns].head())

    if inc > 0.01:
        create_Margin(inc,original_df_up,'up')
        create_Margin(inc,original_df_down,'down')
        classify_up(original_df_up)
        classify_down(original_df_down)
        tmp_df           = tmp_df.append(original_df_up[original_df_up['Group_flag'] == 'neutral_up'])
        tmp_df           = tmp_df.append(original_df_down[original_df_down['Group_flag'] == 'neutral_down'])
        original_df_up   = original_df_up[original_df_up['Group_flag'] == 'up']
        original_df_down = original_df_down[original_df_down['Group_flag'] == 'down']
        #print('tmp\n',tmp_df[list_of_columns].head())
        #print('up\n', original_df_up[list_of_columns].head())
        #print('down\n',original_df_down[list_of_columns].head())
    original_df_down['Margin_base']   = original_df_down['Margin_down']
    original_df_up['Margin_base']   = original_df_up['Margin_up']

    print(len(tmp_df.index))
    print(len(original_df_up.index))
    print(len(original_df_down.index))

Increment = 0.01
264
20
74
Increment = 0.2
345
6
7
Increment = 0.3
353
5
0
Increment = 0.4
356
2
0


In [56]:

tmp_df
#print(or_filt_df[or_filt_df['Group_flag'] =='down'])
#print(or_filt_df[or_filt_df['Group_flag'] =='up'])
#print(or_filt_df[or_filt_df['Group_flag'] =='neutral'])

,item_code,avg_conv_rate,last_price,last_date,avg_unique_views,PE,PE_err,has_enough_stat,pc2,item_skey,...,Margin_err_up,price_down,Delta_price_down,conv_rate_down,conv_rate_err_down,avg_unique_views_down,avg_unique_views_err_down,Margin_down,Margin_err_down,Group_flag
0,1000005727,0.017079,99.99,2021-02-25,170.099034,-1.5183,0.4891,True,40.729,7.148819e+13,...,0.700036,98.995083,42.460145,0.017340,0.000085,171.783304,0.467594,126.479698,0.707961,neutral
2,1000175476,0.055748,83.99,2021-02-25,80.722222,-2.0583,0.2891,True,48.799,1.152606e+14,...,0.324119,83.154286,21.078551,0.056907,0.000165,81.127087,0.126639,97.313460,0.319725,neutral
3,1000176734,0.007736,159.99,2021-02-08,100.555556,-1.7633,0.5678,True,54.119,3.825599e+13,...,0.398081,158.398073,78.988624,0.007873,0.000045,101.820389,0.313098,63.323195,0.408890,neutral
4,1000194074,0.024822,77.99,2021-02-25,97.437774,-1.5244,0.4274,True,43.439,2.425229e+14,...,0.262566,77.213987,21.446703,0.025203,0.000108,98.323096,0.233124,53.146711,0.259760,neutral
5,1000008470,0.020172,89.99,2021-02-13,189.558824,-2.9257,0.9689,True,48.500,6.600352e+13,...,1.057330,89.094585,26.369399,0.020771,0.000201,194.760070,0.784104,106.671466,1.119214,neutral
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
269,1000164539,0.018660,104.99,2021-02-20,81.500000,-2.8948,0.5939,NaN,32.539,NaN,...,NaN,77.778505,32.821088,0.044471,0.007923,128.667688,10.653685,187.803159,36.897652,neutral_down
295,1000117126,0.021216,99.99,2021-02-22,68.712974,-1.9683,0.2992,NaN,29.359,NaN,...,NaN,74.074414,32.888407,0.038291,0.003437,108.685461,7.574290,136.872279,15.553867,neutral_down
36,1000123835,0.011498,269.99,2021-02-13,103.812931,-0.8812,0.0897,NaN,107.620,NaN,...,13.789975,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,neutral_up
242,1000010998,0.007613,269.99,2021-02-16,101.884821,-0.8785,0.1155,NaN,123.260,NaN,...,9.423496,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,neutral_up


In [57]:
tmp_df.set_index(['Group_flag','item_code']).count(level='Group_flag')
#554 neutral, 4 down ,440 up

,avg_conv_rate,last_price,last_date,avg_unique_views,PE,PE_err,has_enough_stat,pc2,item_skey,View_PE,...,Margin_up,Margin_err_up,price_down,Delta_price_down,conv_rate_down,conv_rate_err_down,avg_unique_views_down,avg_unique_views_err_down,Margin_down,Margin_err_down
Group_flag,,,,,,,,,,,,,,,,,,,,,
neutral,264,264,264,264,264,264,264,264,264,264,...,264,264,264,264,264,264,264,264,264,264
neutral_down,74,74,74,74,74,74,0,74,0,74,...,0,0,74,74,74,74,74,74,74,74
neutral_up,18,18,18,18,18,18,0,18,0,18,...,18,18,0,0,0,0,0,0,0,0


In [58]:
tmp_df['Margin_base'] = df_merged['Margin_base']
tmp_df.head()
#df_final_spark.head(400)


,item_code,avg_conv_rate,last_price,last_date,avg_unique_views,PE,PE_err,has_enough_stat,pc2,item_skey,...,Margin_err_up,price_down,Delta_price_down,conv_rate_down,conv_rate_err_down,avg_unique_views_down,avg_unique_views_err_down,Margin_down,Margin_err_down,Group_flag
0,1000005727,0.017079,99.99,2021-02-25,170.099034,-1.5183,0.4891,True,40.729,7.148819e+13,...,0.700036,98.995083,42.460145,0.017340,0.000085,171.783304,0.467594,126.479698,0.707961,neutral
2,1000175476,0.055748,83.99,2021-02-25,80.722222,-2.0583,0.2891,True,48.799,1.152606e+14,...,0.324119,83.154286,21.078551,0.056907,0.000165,81.127087,0.126639,97.313460,0.319725,neutral
3,1000176734,0.007736,159.99,2021-02-08,100.555556,-1.7633,0.5678,True,54.119,3.825599e+13,...,0.398081,158.398073,78.988624,0.007873,0.000045,101.820389,0.313098,63.323195,0.408890,neutral
4,1000194074,0.024822,77.99,2021-02-25,97.437774,-1.5244,0.4274,True,43.439,2.425229e+14,...,0.262566,77.213987,21.446703,0.025203,0.000108,98.323096,0.233124,53.146711,0.259760,neutral
5,1000008470,0.020172,89.99,2021-02-13,189.558824,-2.9257,0.9689,True,48.500,6.600352e+13,...,1.057330,89.094585,26.369399,0.020771,0.000201,194.760070,0.784104,106.671466,1.119214,neutral


In [59]:
#tmp_df[['item_code','avg_conv_rate','last_price','PE','PE_err','has_enough_stat','pc2','price_up','price_down','Margin_base','Margin_up','Margin_err_up','Margin_down','Margin_err_down','Group_flag']].head(400)


In [60]:
tmp_df = tmp_df.sort_index()

In [61]:
tmp_df['Margin_rate_to_base'] = np.where(tmp_df['Group_flag'] == 'neutral', 0.,np.where(tmp_df['Group_flag'] == 'neutral_up', np.log(tmp_df['Margin_up']/tmp_df['Margin_base']),np.log(tmp_df['Margin_down']/tmp_df['Margin_base'])))
#tmp_df['Margin_rate_to_last'] = np.log(tmp_df['Margin_down']/tmp_df['Margin_last'])

In [62]:
list_of_columns_down.append('Margin_err_down')

In [63]:
list_of_columns_up.append('Margin_err_up')

In [64]:
tmp_df[list_of_columns_down]



,item_code,last_date,last_price,price_down,PE,PE_err,View_PE,View_PE_err,Delta_price,pc2,avg_conv_rate,avg_unique_views,avg_unique_views_down,Margin_base,Margin_down,Group_flag,Margin_err_down
0,1000005727,2021-02-25,99.99,98.995083,-1.5183,0.4891,-0.9853,0.2722,43.296210,40.729,0.017079,170.099034,171.783304,125.781343,126.479698,neutral,0.707961
1,1000016028,2021-02-25,139.99,114.614118,-2.3105,0.5174,-1.6602,0.2891,65.208655,52.430,0.016290,26.446154,36.860877,28.091607,41.828469,neutral_down,4.958264
2,1000175476,2021-02-25,83.99,83.154286,-2.0583,0.2891,-0.5003,0.1561,21.780832,48.799,0.055748,80.722222,81.127087,98.015502,97.313460,neutral,0.319725
3,1000176734,2021-02-08,159.99,158.398073,-1.7633,0.5678,-1.2500,0.3075,80.326378,54.119,0.007736,100.555556,101.820389,62.484149,63.323195,neutral,0.408890
4,1000194074,2021-02-25,77.99,77.213987,-1.5244,0.4274,-0.9045,0.2371,22.098815,43.439,0.024822,97.437774,98.323096,53.448592,53.146711,neutral,0.259760
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
353,1000196399,2021-02-25,84.99,84.144335,-2.9264,0.5390,-1.2779,0.3463,14.710168,56.710,0.032453,67.459459,68.327056,32.204333,31.964583,neutral,0.204784
354,1000194221,2021-01-08,159.99,NaN,-1.5819,0.3958,-0.9023,0.2801,31.096378,103.349,0.025424,40.125730,NaN,31.723225,NaN,neutral_up,NaN
355,1000109909,2021-02-25,239.99,237.602060,-1.8804,0.3834,-0.9106,0.1694,125.582269,76.090,0.012308,55.833333,56.344074,86.302682,87.327262,neutral,0.366038
356,1000123847,2021-02-12,149.99,122.801426,-3.5520,0.9811,-1.3020,0.2721,77.652017,48.390,0.011562,80.744322,104.761636,72.495808,135.079443,neutral_down,27.505779


In [65]:
tmp_df[list_of_columns_up]

,item_code,last_date,last_price,price_up,PE,PE_err,View_PE,View_PE_err,Delta_price,pc2,avg_conv_rate,avg_unique_views,avg_unique_views_up,Margin_base,Margin_up,Group_flag,Margin_err_up
0,1000005727,2021-02-25,99.99,100.994916,-1.5183,0.4891,-0.9853,0.2722,43.296210,40.729,0.017079,170.099034,168.431278,125.781343,125.064008,neutral,0.700036
1,1000016028,2021-02-25,139.99,NaN,-2.3105,0.5174,-1.6602,0.2891,65.208655,52.430,0.016290,26.446154,NaN,28.091607,NaN,neutral_down,NaN
2,1000175476,2021-02-25,83.99,84.834114,-2.0583,0.2891,-0.5003,0.1561,21.780832,48.799,0.055748,80.722222,80.319378,98.015502,98.650935,neutral,0.324119
3,1000176734,2021-02-08,159.99,161.597926,-1.7633,0.5678,-1.2500,0.3075,80.326378,54.119,0.007736,100.555556,99.306434,62.484149,61.649268,neutral,0.398081
4,1000194074,2021-02-25,77.99,78.773813,-1.5244,0.4274,-0.9045,0.2371,22.098815,43.439,0.024822,97.437774,96.560423,53.448592,53.720852,neutral,0.262566
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
353,1000196399,2021-02-25,84.99,85.844164,-2.9264,0.5390,-1.2779,0.3463,14.710168,56.710,0.032453,67.459459,66.602880,32.204333,32.385151,neutral,0.207479
354,1000194221,2021-01-08,159.99,195.412227,-1.5819,0.3958,-0.9023,0.2801,31.096378,103.349,0.025424,40.125730,33.500413,31.723225,37.778585,neutral_up,3.663655
355,1000109909,2021-02-25,239.99,242.401940,-1.8804,0.3834,-0.9106,0.1694,125.582269,76.090,0.012308,55.833333,55.327223,86.302682,85.281825,neutral,0.357464
356,1000123847,2021-02-12,149.99,NaN,-3.5520,0.9811,-1.3020,0.2721,77.652017,48.390,0.011562,80.744322,NaN,72.495808,NaN,neutral_down,NaN


In [66]:
tmp_df.to_csv('../raw_data/Excel_files/Margin_result_View_PE.csv')

In [67]:
tmp_df[['item_code','avg_unique_views','avg_unique_views_up','avg_unique_views_down']].dropna()

,item_code,avg_unique_views,avg_unique_views_up,avg_unique_views_down
0,1000005727,170.099034,168.431278,171.783304
2,1000175476,80.722222,80.319378,81.127087
3,1000176734,100.555556,99.306434,101.820389
4,1000194074,97.437774,96.560423,98.323096
5,1000008470,189.558824,184.496481,194.760070
...,...,...,...,...
350,1000016020,543.000000,537.187567,548.875324
351,1000174112,327.649206,321.865110,333.537247
353,1000196399,67.459459,66.602880,68.327056
355,1000109909,55.833333,55.327223,56.344074


In [68]:
tmp_df_only_change=tmp_df[tmp_df['Group_flag']!='neutral']

In [69]:
tmp_df_only_change['Margin_rate_to_base'].mean()

0.45208630641681496

In [70]:
tmp_df_only_change['Margin_rate_to_base'].nlargest(10)


307    1.008573
269    0.796379
248    0.785374
118    0.770244
297    0.767550
58     0.754146
85     0.746175
162    0.714835
242    0.707415
182    0.701022
Name: Margin_rate_to_base, dtype: float64

In [71]:
tmp_df_only_change['Margin_rate_to_base'].median()

0.4811474967913334